In [1]:
import requests
import pandas as pd
from functools import *
import numpy as np
import nltk
%load_ext autoreload
%autoreload 2
import recip as rr
import dill
import submatcher as sm

In [2]:
inp = ["flour","butter","milk", "egg", "cheese"]

In [3]:
allr = rr.GetRecipies(inp)

#add self.get() to __init__ and bottom 4 lines are replaced - just saving an API call for now
dbfile = open('resp.pkl', 'rb') 
dbfile.seek(0)
allr.response =  dill.load(dbfile)

allr.process()

In [4]:
import json
with open('test.json', 'w') as f:
    json.dump(allr.to_json(), f,skipkeys = True)

NameError: name 'r' is not defined

In [ ]:
# r = allr.recser.iloc[0]
# dbrfile = open('respr.pkl', 'rb') 
# r.response =  dill.load(dbrfile)
# r.setVars()

In [ ]:
# r.to_json()